In [1]:
from typing import TypedDict, Annotated, Sequence
from typing_extensions import TypedDict

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, FunctionMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain import hub
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Your further code goes here

In [2]:
import re
import time
import pandas as pd
from pprint import pprint

from gsheet_handler import GSheetHandler
import importlib
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')


no docx
no docx


In [3]:
# main.py
import json
import os
import telebot

# Загрузка конфигурации из JSON-файла
with open('config.json') as config_file:
    config = json.load(config_file)

# Установка переменных окружения
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="nlmk_bot"

In [4]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [5]:
from langchain import hub
prompt = hub.pull("mlenparrot/nlmk_aiparse")
prompt

StructuredPrompt(input_variables=['text'], metadata={'lc_hub_owner': 'mlenparrot', 'lc_hub_repo': 'nlmk_aiparse', 'lc_hub_commit_hash': '30c7ce342714d1b98d3ddfab0efc20257eb69bfe6ed3f1da47178d768a99475e'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Нужен список [] ВСЕХ новостей.\n\nlink: Ссылка на источник \nnews_data: Дата новости \nnews_topic: Заголовок новости\nnews_text: Текст новости\n\nneed JSON output')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))], schema_={'type': 'object', 'title': 'extract', 'required': ['news_list'], 'properties': {'news_list': {'type': 'object', 'properties': {'link': {'type': 'string', 'description': 'Ссылка на источник'}, 'news_data': {'type': 'string', 'format': 'date', 'description': 'Дата новости'}, 'news_text': {'type': 'string', 'description': 'Текст новости'}, 'news_topic': {'type': 'string', 'description': 'Заголовок новости'}}, 'description': 'Нужен 

In [6]:
import re

def remove_repeated_newlines(text):
    # Удаляем повторяющиеся переносы строки
    cleaned_text = re.sub(r'\n+', '\n', text)
    return cleaned_text

# Пример использования
text_with_repeated_newlines = """Это пример текста с
многими

повторяющимися

переносами строки.
"""

cleaned_text = remove_repeated_newlines(text_with_repeated_newlines)
print(cleaned_text)

Это пример текста с
многими
повторяющимися
переносами строки.



In [7]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright, TimeoutError
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datetime import datetime
import pandas as pd
from tqdm.asyncio import tqdm_asyncio
import tiktoken

nest_asyncio.apply()

# Function to count tokens
def count_tokens(text):
    encoding = tiktoken.get_encoding("cl100k_base")
    tokens = encoding.encode(text)
    return len(tokens)

# Function to calculate cost
def calculate_cost(tokens, model):
    if model == "gpt-4o":
        cost_per_token_input = 5 / 1_000_000  # Cost per token for GPT-4o input
        cost_per_token_output = 15 / 1_000_000  # Cost per token for GPT-4o output
        cost_input = tokens * cost_per_token_input
        cost_output = tokens * cost_per_token_output
        return cost_input + cost_output
    elif model == "gpt-3.5-turbo":
        cost_per_token = 0.002 / 1000  # Placeholder cost per token for GPT-3.5-turbo
        return tokens * cost_per_token

def save_html_dump(html_content, url, prefix=""):
    filename = f"{prefix}dump_{url.replace('https://', '').replace('/', '_')}.html"
    with open(filename, "w", encoding="utf-8") as file:
        file.write(html_content)

async def extract_text_and_links_from_page(url, max_wait_time=60):
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
            page = await context.new_page()

            active_requests = set()

            def on_request(request):
                active_requests.add(request)

            def on_request_finished(request):
                active_requests.discard(request)

            def on_request_failed(request):
                active_requests.discard(request)

            page.on("request", on_request)
            page.on("requestfinished", on_request_finished)
            page.on("requestfailed", on_request_failed)

            try:
                await page.goto(url, wait_until='domcontentloaded', timeout=30000)
            except TimeoutError:
                html_content = await page.content()
                save_html_dump(html_content, url, prefix="timeout_")
                await browser.close()
                return None, f"TimeoutError navigating to {url}"

            start_time = datetime.now()
            while active_requests and (datetime.now() - start_time).seconds < max_wait_time:
                print(f"Waiting for {len(active_requests)} active requests...")
                await page.wait_for_timeout(1000)  # Wait for 1 second before checking again
                html_content = await page.content()
                save_html_dump(html_content, url)

            if active_requests:
                print(f"Timeout waiting for active requests to complete for {url}")

            html_content = await page.content()
            save_html_dump(html_content, url)
            await browser.close()
            return html_content, None
    except Exception as e:
        return None, str(e)

async def process_url(url):
    html_content, error = await extract_text_and_links_from_page(url)
    if html_content:
        try:
            soup = BeautifulSoup(html_content, 'html.parser')
            base_url = url
            
            for a in soup.find_all('a', href=True):
                absolute_url = urljoin(base_url, a['href'])
                a.insert_after(f'({absolute_url})')
            
            text_content = soup.get_text()
            current_date = datetime.now().strftime("%Y-%m-%d")
            full_text = f"Today's date: {current_date}\n\n{text_content}"
            
            # Calculate tokens and costs
            tokens = count_tokens(full_text)
            cost_gpt4o = calculate_cost(tokens, "gpt-4o")
            cost_gpt35 = calculate_cost(tokens, "gpt-3.5-turbo")
            
            return {
                "url": url, 
                "content": full_text, 
                "error": None, 
                "tokens": tokens, 
                "cost_gpt4o": cost_gpt4o, 
                "cost_gpt35": cost_gpt35
            }
        except Exception as e:
            tokens = count_tokens(str(e))
            cost_gpt4o = calculate_cost(tokens, "gpt-4o")
            cost_gpt35 = calculate_cost(tokens, "gpt-3.5-turbo")
            return {
                "url": url, 
                "content": None, 
                "error": str(e), 
                "tokens": tokens, 
                "cost_gpt4o": cost_gpt4o, 
                "cost_gpt35": cost_gpt35
            }
    else:
        tokens = count_tokens(error)
        cost_gpt4o = calculate_cost(tokens, "gpt-4o")
        cost_gpt35 = calculate_cost(tokens, "gpt-3.5-turbo")
        return {
            "url": url, 
            "content": None, 
            "error": error, 
            "tokens": tokens, 
            "cost_gpt4o": cost_gpt4o, 
            "cost_gpt35": cost_gpt35
        }

async def main(urls):
    results = []
    for result in tqdm_asyncio.as_completed([process_url(url) for url in urls]):
        results.append(await result)
    return results


In [8]:
import re
import time
import pandas as pd
from pprint import pprint

from gsheet_handler import GSheetHandler
import importlib
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

no docx


In [9]:
urls_raw = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit#gid=2038294867', 
    0)
urls_raw

,site,rss_feed,autopars RSS,AI parse,handmade parser,in_specificaion,relevant,
0,https://www.theverge.com,https://www.theverge.com/ai-artificial-intelli...,1,,+,+,+,
1,https://techcrunch.com,https://techcrunch.com/category/artificial-int...,1,,-,+,+,
2,https://www.forbes.com/innovation/?sh=49e63dc3...,https://www.forbes.com/innovation/feed/,1,,-,+,+,
3,https://venturebeat.com/category/ai/,http://feeds.feedburner.com/venturebeat/SZYF,1,,-,+,+,
4,https://www.digitaltrends.com/,https://www.itprotoday.com/rss.xml,1,,-,+,+,
...,...,...,...,...,...,...,...,...
70,www.bnkomi.ru,,,,+,-,-,
71,https://kapi.bar,,,,+,-,-,
72,https://kostroma.today,,,,+,-,-,
73,www.metalbulletin.ru,,,,+,-,+,


In [10]:
urls = urls_raw[urls_raw['AI parse'] == '1'][['site']].reset_index(drop=True)
urls

,site
0,https://edtechmagazine.com/
1,https://www.edsurge.com/
2,https://www.rbc.ru
3,https://severstal.com/rus/media/
4,https://social.evraz.com/news/
5,https://omk.ru/press/
6,https://omk.ru/press/news/
7,https://evraz.market/about/news/
8,https://www.cbinsights.com/research/
9,https://www.bcg.com/


In [11]:
urls['site'].tolist()

['https://edtechmagazine.com/',
 'https://www.edsurge.com/',
 'https://www.rbc.ru',
 'https://severstal.com/rus/media/',
 'https://social.evraz.com/news/',
 'https://omk.ru/press/',
 'https://omk.ru/press/news/',
 'https://evraz.market/about/news/',
 'https://www.cbinsights.com/research/',
 'https://www.bcg.com/',
 'https://www.bain.com/',
 'https://www2.deloitte.com/global/en.html',
 'https://www.pwc.com',
 'https://www.ey.com/en_gl',
 'https://kpmg.com/xx/en/home.html',
 'https://www.gartner.com/en',
 'https://www.forrester.com/bold',
 'https://datainsight.ru/',
 'https://trends.rbc.ru/trends/']

In [12]:
results = await main(urls['site'].tolist())

  0%|          | 0/19 [00:00<?, ?it/s]

Waiting for 4 active requests...
Waiting for 6 active requests...
Waiting for 28 active requests...
Waiting for 5 active requests...
Waiting for 5 active requests...
Waiting for 2 active requests...
Waiting for 4 active requests...


  5%|▌         | 1/19 [00:50<15:10, 50.59s/it]

Waiting for 2 active requests...
Waiting for 1 active requests...


 11%|█         | 2/19 [00:56<06:54, 24.37s/it]

...

 89%|████████▉ | 17/19 [01:58<00:15,  7.77s/it]

Waiting for 41 active requests...
Waiting for 6 active requests...
Waiting for 41 active requests...
Waiting for 6 active requests...
Waiting for 42 active requests...
Waiting for 6 active requests...
Waiting for 42 active requests...
Waiting for 6 active requests...
Waiting for 43 active requests...
Waiting for 6 active requests...
Waiting for 43 active requests...
Waiting for 7 active requests...
Waiting for 44 active requests...
Waiting for 6 active requests...
Waiting for 45 active requests...
Waiting for 6 active requests...
Waiting for 45 active requests...
Waiting for 6 active requests...
Waiting for 6 active requests...
Waiting for 46 active requests...
Timeout waiting for active requests to complete for https://www.rbc.ru
Waiting for 46 active requests...


 95%|█████████▍| 18/19 [02:10<00:08,  8.99s/it]

Timeout waiting for active requests to complete for https://evraz.market/about/news/


100%|██████████| 19/19 [02:11<00:00,  6.91s/it]


In [13]:
df = pd.DataFrame(results)
df.to_csv('web_content.csv', index=False)
df

,url,content,error,tokens,cost_gpt4o,cost_gpt35
0,https://edtechmagazine.com/,Today's date: 2024-06-12\n\n\n\n\n\n\n\n\nK12 ...,None,110,0.00220,0.000220
1,https://severstal.com/rus/media/,Today's date: 2024-06-12\n\n\n \n\n\n\n\n\n\n\...,None,19859,0.39718,0.039718
2,https://www.edsurge.com/,Today's date: 2024-06-12\n\n\nEdSurge | Educat...,None,4417,0.08834,0.008834
3,https://datainsight.ru/,None,TimeoutError navigating to https://datainsight...,11,0.00022,0.000022
4,https://www.bcg.com/,Today's date: 2024-06-12\n\n\n\n\n\n\n\n\n\n\n...,None,10378,0.20756,0.020756
5,https://www.forrester.com/bold,Today's date: 2024-06-12\n\n\n\n\n\n\n\n\n\n\n...,None,5907,0.11814,0.011814
6,https://www.bain.com/,Today's date: 2024-06-12\n\n\nGlobal Managemen...,None,10688,0.21376,0.021376
7,https://www2.deloitte.com/global/en.html,Today's date: 2024-06-12\n\n\n\n\nDeloitte | A...,None,7978,0.15956,0.015956
8,https://www.pwc.com,Today's date: 2024-06-12\n\n\n\n\n\n\n\n\n\n\n...,None,10716,0.21432,0.021432
9,https://www.ey.com/en_gl,Today's date: 2024-06-12\n\n\n\nEY - Global | ...,None,12318,0.24636,0.024636


In [14]:
#results

In [15]:
df.keys()

Index(['url', 'content', 'error', 'tokens', 'cost_gpt4o', 'cost_gpt35'], dtype='object')

In [16]:
df_ggl = df.copy()
df_ggl['content'] = df_ggl['content'].fillna('').astype(str)
df_ggl['content'] = df_ggl['content'].apply(remove_repeated_newlines)
df_ggl['content'] = df_ggl['content'].str[:40000]

In [22]:
ggl.update_gsheet_with_df(df_ggl,
                          'https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit#gid=0', 
                        2,
                          integer_columns=[3],
                          float_columns=[4,5])